# DB Connection

In [266]:
import cassandra 
print (cassandra.__version__)
import pandas
print(pandas.__version__)
import pandas as pd
import uuid

3.25.0
1.0.1


In [267]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.concurrent import execute_concurrent_with_args
# cloud_config1 = {
#         'secure_connect_bundle': '../secure-connect-capstone-project.zip'
# }
# auth_provider1 = PlainTextAuthProvider('JJvGOjuOPszNbfeoOwEinlHI', 'ZM-OR5KZAIMzqksmn01xobmZW4cNRUiIft0SnYuhW966f++ILhp+2Mq,AErDqKAF7Y0AF4ReSYLjTCgsLCqRJiL3+8uxJMoS-K6qMtM1vEmgi5IT5c62OvBJkhxBzq5T')
# cluster1 = Cluster(cloud=cloud_config1, auth_provider=auth_provider1)
# #connect to keyspace_name
# session1 = cluster1.connect('iac689')

cloud_config2= {
        'secure_connect_bundle': '../secure-connect-capstone-project2.zip'
}
auth_provider2 = PlainTextAuthProvider('OZTxZCgzatSukjKBoWpZimZs', 'cfph4pZMPt-REY_B0BTCZ-qxZfwgpIeZA2T69DIzccYlZ2I1+8gKKk6Ruk+G+70.CyFNoZymWef1Eo1KJZk,SjYbo9SgRZwY97JfG6J8uFLvNEJ8,nCjMbcGz.wgmlJC')
cluster2 = Cluster(cloud=cloud_config2, auth_provider=auth_provider2)
#connect to keyspace_name
session2 = cluster2.connect('iac689')

In [271]:
query = "SELECT * FROM system_schema.keyspaces;"
df = pd.DataFrame(list(session2.execute(query)))
df

,keyspace_name,durable_writes,graph_engine,replication
0,iac689,True,None,"[class, us-east1]"
1,system_auth,True,None,"[class, us-east1]"
2,system_schema,False,None,[class]
3,data_endpoint_auth,True,None,"[class, us-east1]"
4,datastax_sla,True,None,"[class, us-east1]"
5,system,False,None,[class]
6,system_traces,True,None,"[class, us-east1]"


In [272]:
query = "SELECT * FROM system_schema.tables WHERE keyspace_name='iac689';"
df = pd.DataFrame(list(session2.execute(query)))
df

,keyspace_name,table_name,additional_write_policy,bloom_filter_fp_chance,caching,cdc,comment,compaction,compression,crc_check_chance,...,flags,gc_grace_seconds,id,max_index_interval,memtable_flush_period_in_ms,min_index_interval,nodesync,read_repair,read_repair_chance,speculative_retry
0,iac689,data_2,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,a68727d0-7f1c-11ec-bb17-45ae5e7b1bda,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
1,iac689,kmeans,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,95ef9290-8944-11ec-bb17-45ae5e7b1bda,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
2,iac689,newdata,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,3571f450-8dcf-11ec-a016-ef0b22327c1e,2048,0,128,None,BLOCKING,0.0,99PERCENTILE


In [273]:
query = "SELECT * FROM system_schema.columns WHERE keyspace_name='iac689' and table_name = 'data_1';"
df = pd.DataFrame(list(session2.execute(query)))
df

""


# DB_Insert

In [274]:
import pandas as pd
import glob

In [275]:
path_list = glob.glob('../../cap_data2/*.csv')
df_list = []
for i in path_list:
    df_list.append(pd.read_csv(i))

/Users/vathanahim/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [276]:
for i in range(len(df_list)):
    if (i < 5):
        df_list[i]['time'] = pd.to_datetime(df_list[i]['time']).dt.date
        df_list[i]['truck_name'] = df_list[i]['truck_name'].str.replace(r"[a-zA-Z|-]","")
        df_list[i]['truck_name'] = 'abc-'+df_list[i]['truck_id'].astype(str)
    else:
        df_list[i]['start_time'] = pd.to_datetime(df_list[i]['start_time']).dt.date
        df_list[i]['truck_name'] = df_list[i]['truck_name'].str.replace(r"[a-zA-Z|-]","")
        df_list[i]['truck_name'] = 'abc-'+df_list[i]['truck_id'].astype(str)
       

In [172]:

#Insert for the last dataframe into a different table (kind of slow)
# query = "INSERT INTO data_1 (uid, active, reading_id, start_mileage, start_time, truck_id, truck_name, type) VALUES (?,?,?,?,?,?,?,?)"
# prepared = session1.prepare(query)
# for i in df.itertuples():
#     print(i.truck_id)
#     session1.execute(prepared, (uuid.uuid1(), i.active, i.reading_id, i.start_mileage, i.start_time, i.truck_id, i.truck_name, i.not_active))

In [282]:
df_engine_speed = df_list[0].copy()
df_altitude = df_list[1].copy().dropna()
df_inclination = df_list[2].copy()
df_wheel_based_speed = pd.concat([df_list[3],df_list[4]])

In [283]:
inclination_hist = [-20, -18,-15,-13, -11.5,-10.5,-9.5,-8.5,-7.5,-6.5,-5.5,-4.5,-3.5,-2.5,-1.5,-0.5,0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5,13.0,15.0,18.0,20.0]
altitude_hist = [i for i in range(500, 7000, 500)]
vehicle_speed_hist = [0,5,12,19,26,33,40,46,53,60,68,75,82,89,96,103,110,116,123,130]
engine_speed_hist = [i for i in range(0,12500, 500)]

In [284]:
def helper(df, bin_val:list):
#     df= df.pivot_table(index=['time', 'truck_id','type'], columns='bin_num',
#                      values='value').reset_index()
    df = df[df.bin_num !='start_mileage']
    num = len(df['bin_num'].unique())
    bin_val = bin_val[:num]
    print(len(bin_val),num)
    new_val = []
    for i in df['bin_num']:
        n = int(i.split('_')[-1])
        new_val.append(bin_val[n])
    df['new_val'] = new_val
    df['final_val'] = (df['value']*df['new_val'])
    return df

In [285]:
df_engine_speed = helper(df_engine_speed, engine_speed_hist)
df_inclination = helper(df_inclination, inclination_hist)
df_wheel_based_speed = helper(df_wheel_based_speed, vehicle_speed_hist)

24 24


/Users/vathanahim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/vathanahim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


31 31
19 19


In [286]:
df_altitude = df_altitude.rename(columns={'value':'final_val'})
df_altitude

,time,truck_name,truck_id,type,reading_id,bin_num,final_val
0,2021-11-17,abc-242,242,Time(hours) at different Altitudes,86021,col_0,0.000000
1,2021-11-17,abc-242,242,Distance(miles) at different Altitudes,86021,col_0,0.000000
2,2021-11-17,abc-242,242,Time(hours) at different Altitudes,86021,col_0,0.000000
3,2021-11-17,abc-242,242,Distance(miles) at different Altitudes,86021,col_0,0.000000
4,2021-01-04,abc-294,294,Time(hours) at different Altitudes,86022,col_0,0.772778
...,...,...,...,...,...,...,...
95467,2021-02-15,abc-277,277,Distance(miles) at different Altitudes,130151,col_0,0.000000
95468,2021-02-17,abc-277,277,Time(hours) at different Altitudes,130152,col_0,0.000000
95469,2021-02-17,abc-277,277,Distance(miles) at different Altitudes,130152,col_0,0.000000
95470,2021-02-17,abc-277,277,Time(hours) at different Altitudes,130152,col_0,0.000000


In [330]:
df_list = [df_engine_speed, df_altitude, df_inclination, df_wheel_based_speed]
df = pd.concat(df_list)


In [331]:
df = df.pivot_table(index=['time', 'truck_id'], columns='type',
                     values='final_val',aggfunc='mean').reset_index()


In [332]:
df['uuid'] = df.index.to_series().map(lambda x: uuid.uuid4())
df

type,time,truck_id,Distance(miles) at WheelBasedVehicleSpeed(mph),Distance(miles) at different Altitudes,Distance(miles) at different Engine Speeds (rpms),Distance(miles) at different RoadInclination(%),Time(hours) at WheelBasedVehicleSpeed(mph),Time(hours) at different Altitudes,Time(hours) at different Engine Speeds (rpms),Time(hours) at different RoadInclination(%),uuid
0,2020-12-28,286,-1.898115e+06,162463.500000,6.202083e+05,1339.112903,1.792842,1.815138,47.531875,0.000184,baab0bec-2615-4510-afb9-ccf8dd66969f
1,2020-12-29,286,-1.808075e+06,144921.500000,1.091979e+06,-789.274194,1.813446,1.781807,48.625611,-0.002753,f1019a50-2cc9-4fd5-8dca-9816ad785b6f
2,2020-12-30,286,-1.600186e+06,119689.500000,1.805208e+05,587.849462,1.383603,1.565835,40.549742,0.002549,b056b8b2-2fd3-4ab1-a25d-7ccc3305213f
3,2020-12-31,286,-1.647074e+06,148580.433333,-8.194444e+04,1372.231183,1.446769,1.470555,37.285868,0.002567,5e3fdb3d-bdb1-451d-808a-516ae6c95ec0
4,2021-01-01,207,0.000000e+00,0.000000,0.000000e+00,0.000000,3.031754,0.001250,41.710093,-0.019326,4b943527-2570-4c97-822d-b71ee7e5f247
...,...,...,...,...,...,...,...,...,...,...,...
4542,2022-01-08,294,6.114824e+04,599411.000000,2.362917e+06,NaN,2.105701,0.858795,45.623099,NaN,8b0262f5-772d-4bf1-9526-b62c936b7140
4543,2022-01-08,296,3.134737e+05,0.000000,3.685158e+06,3947.306210,4.108464,0.000000,53.315971,0.050108,29a49321-68ea-4ac6-800d-903381e43f02
4544,2022-01-09,208,8.796911e+05,NaN,8.310508e+06,16026.783871,10.394748,NaN,108.892445,0.309436,204b2e72-ae61-4357-a080-fb4df9169a19
4545,2022-01-09,209,0.000000e+00,135474.000000,0.000000e+00,0.000000,0.401286,0.255278,5.520833,0.029919,ee6d5458-1c79-432f-bf20-b2189079e56b


In [333]:
df.describe()

type,truck_id,Distance(miles) at WheelBasedVehicleSpeed(mph),Distance(miles) at different Altitudes,Distance(miles) at different Engine Speeds (rpms),Distance(miles) at different RoadInclination(%),Time(hours) at WheelBasedVehicleSpeed(mph),Time(hours) at different Altitudes,Time(hours) at different Engine Speeds (rpms),Time(hours) at different RoadInclination(%)
count,4547.000000,4.547000e+03,3.543000e+03,4.546000e+03,3.697000e+03,4547.000000,3543.000000,4546.000000,3697.000000
mean,258.192435,5.904024e+05,6.644728e+05,-1.007648e+07,9.042787e+04,5.588392,1.133195,62.053591,0.044946
std,35.043084,5.040838e+07,9.948156e+05,1.842733e+09,1.362355e+07,5.321105,1.353892,61.083809,0.053123
min,201.000000,-1.471428e+09,0.000000e+00,-7.580179e+10,-3.635681e+08,0.001823,0.000000,0.000000,-0.280778
25%,221.000000,3.858464e+04,1.080975e+02,7.202732e+05,6.297234e+02,1.319545,0.021667,15.944286,0.006910
50%,274.000000,2.365934e+05,2.793622e+05,2.906849e+06,2.912236e+03,3.947383,0.612253,44.037065,0.032606
75%,290.000000,5.951738e+05,9.243595e+05,6.748139e+06,6.342094e+03,8.754609,1.915750,99.027049,0.069507
max,297.000000,2.286124e+09,1.674670e+07,7.505267e+10,3.462059e+08,91.141168,13.541700,958.674500,0.611095


In [334]:
df['Distance(miles) at WheelBasedVehicleSpeed(mph)'] = abs(df['Distance(miles) at WheelBasedVehicleSpeed(mph)'])
df['Distance(miles) at different Engine Speeds (rpms)'] = abs(df['Distance(miles) at different Engine Speeds (rpms)'])
df['Time(hours) at different Engine Speeds (rpms)']= abs((df['Time(hours) at different Engine Speeds (rpms)']-df['Time(hours) at different Engine Speeds (rpms)'].min())/(df['Time(hours) at different Engine Speeds (rpms)'].max()-df['Time(hours) at different Engine Speeds (rpms)'].min()))
df['Time(hours) at different RoadInclination(%)'] = abs(df['Time(hours) at different RoadInclination(%)'])

In [335]:
l = []
for i in df[['uuid','time','Distance(miles) at different Altitudes','Distance(miles) at different Engine Speeds (rpms)','Distance(miles) at different RoadInclination(%)', 'Distance(miles) at WheelBasedVehicleSpeed(mph)','Time(hours) at different Altitudes', 'Time(hours) at different Engine Speeds (rpms)','Time(hours) at different RoadInclination(%)','Time(hours) at WheelBasedVehicleSpeed(mph)','truck_id']].columns:
    l.append(df[i].tolist())



In [336]:
query = """insert into newdata (ui , datel , distance_at_diff_altitude , distance_at_diff_engine_speeds , distance_at_diff_roadinclination , distance_at_wheelbased_vehicle_speed , time_at_diff_altitude , time_at_diff_engine_speeds , time_at_diff_roadinclination , time_at_wheelbased_vehicle_speed , truck_id)
values (?,?,?,?,?,?,?,?,?,?,?);"""
prepared = session2.prepare(query)
execute_concurrent_with_args(session2, prepared, zip(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10]))

[ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f8d01369dd0>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f8d0139db90>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f8d01369950>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f8d013bb750>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f8d013f94d0>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f8d01387dd0>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f8d013ad050>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f8d013ad290>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f8d01410710>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object a

In [45]:
query = "select * from iac689.data_1;"
df = pd.DataFrame(list(session1.execute(query)))
df

,uid,active,reading_id,start_mileage,start_time,truck_id,truck_name,type
0,1b644c6e-7f03-11ec-99b7-acde48001122,5400.000000,103995,15906600.0,2021-02-19,205,abc-205,Distance(miles) with CC active
1,1ba064e2-7f03-11ec-99b7-acde48001122,7340.000000,103997,15912000.0,2021-02-19,205,abc-205,Distance(miles) with CC active
2,d847cb7c-7f02-11ec-99b7-acde48001122,11815.000000,103358,12433600.0,2021-01-13,205,abc-205,Distance(miles) with CC active
3,6a9d1424-7f02-11ec-99b7-acde48001122,0.021667,99698,17789500.0,2021-01-11,284,abc-284,Time(hours) with CC Active
4,f6d2f940-7f02-11ec-99b7-acde48001122,0.138056,103646,14102000.0,2021-02-01,205,abc-205,Time(hours) with CC Active
...,...,...,...,...,...,...,...,...
8525,eb7bfd62-7f02-11ec-99b7-acde48001122,0.000000,103539,13469200.0,2021-01-25,205,abc-205,Distance(miles) with CC active
8526,fe85cb18-7f02-11ec-99b7-acde48001122,0.115278,103720,14719100.0,2021-02-05,205,abc-205,Time(hours) with CC Active
8527,17da6c18-7f03-11ec-99b7-acde48001122,0.254722,103961,15560800.0,2021-02-16,205,abc-205,Time(hours) with CC Active
8528,5300048e-7f02-11ec-99b7-acde48001122,0.130556,86044,8068540.0,2021-02-15,294,abc-294,Time(hours) with CC Active


In [44]:
query = "select * from iac689.data_2;"
df = pd.DataFrame(list(session2.execute(query)))
df

,ui,datel,distance_at_diff_altitude,distance_at_diff_engine_speeds,distance_at_diff_roadinclination,distance_at_wheelbased_vehicle_speed,time_at_diff_altitude,time_at_diff_engine_speeds,time_at_diff_roadinclination,time_at_wheelbased_vehicle_speed,truck_id
0,1c9a40f7-28bd-448b-837f-7e6f26867cee,2021-01-12,2.468247e+05,7.139896e+05,5.578044e+05,9.648508e+05,0.318268,7.137108e+05,5.575866e+05,9.644741e+05,284
1,eed85ca4-db7c-42a3-b4f6-46660ac6f5ab,2021-10-01,0.000000e+00,1.087964e+07,8.499720e+06,1.470222e+07,0.000000,1.087643e+07,8.497214e+06,1.469788e+07,296
2,69b49f87-6525-4203-8168-a406da377443,2021-11-18,NaN,1.010224e+07,7.892374e+06,1.365167e+07,NaN,1.010193e+07,7.892130e+06,1.365125e+07,282
3,375516c0-521f-4411-b83a-74f84ab454be,2021-05-12,5.921890e+05,5.071094e+05,NaN,6.852830e+05,1.227917,5.051501e+05,NaN,6.826352e+05,294
4,3f15f13f-62e0-435a-910b-317c3fda47ac,2021-01-25,NaN,2.643425e+07,2.065176e+07,3.572196e+07,NaN,2.643222e+07,2.065017e+07,3.571922e+07,279
...,...,...,...,...,...,...,...,...,...,...,...
4542,2d46a503-3581-40eb-9ad9-124e7a362185,2021-07-27,5.230573e+05,8.439053e+05,NaN,1.140412e+06,0.531806,8.434997e+05,NaN,1.139864e+06,205
4543,9932ae23-5a33-42f1-9a87-ec803a18c325,2021-08-06,8.052310e+05,8.942562e+05,NaN,1.208454e+06,4.340280,8.896362e+05,NaN,1.202211e+06,294
4544,c0b4c700-96ab-438e-a272-e45ba97a3974,2021-06-03,2.373810e+06,2.268493e+05,NaN,3.065532e+05,2.066203,2.259348e+05,NaN,3.053172e+05,292
4545,e29ed179-f997-429f-b592-8158bd7b2030,2021-10-11,NaN,5.238979e+06,4.092952e+06,7.079684e+06,NaN,5.232000e+06,4.087500e+06,7.070270e+06,208
